In [74]:
import os
import numpy as np
from urllib.parse import urlencode 
import pandas as pd 
from IPython.display import Image, display, HTML
import astropy.io.votable.tree 
from astropy.io.votable.tree import VOTableFile, Resource, Table, Field
from astropy import units as u 
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta
from cadcutils import net
import cadctap
from cadctap.core import main_app
from cadctap import CadcTapClient
#from astropy.table import Table
from six import  BytesIO
import glue_jupyter as gj

#
#
#
def create_a_new_votable(field_dictionary,df):
    print("Creating the votable file for the tmp table.")
    num_row = df.shape[0]  #get total number of rows in the dataframe  
    votable = VOTableFile()
    resource = Resource()
    votable.resources.append(resource)
    table = Table(votable)
    resource.tables.append(table)
    list =[]
    for key, value in field_dictionary.items():
        if(len(value)==2):
            list.append(Field(votable, name = key, datatype= value[0], arraysize = value[1]))
        elif(len(value)==3):
            list.append(Field(votable, name = key, datatype= value[0], arraysize = value[1], xtype = value[2], config={'version_1_2_or_later':True}))#hacky way 
        else:
            list.append(Field(votable, name = key, datatype= value))
                
    table.fields.extend(list)
    table.create_arrays(num_row)
    ### replace the NaN value in satrt_date and end_date with 0.0
    
    for i in range(0,num_row):
        sn = sn_data.iloc[i]
        table.array[i] = (str(sn["event"]),str(sn["alias"]), str(sn["catalog"]), str(sn["claimedtype"]), sn["dec"],
                            str(sn["maxdate"]), str(sn["name"]),sn["ra" ],
                          sn["start_date"],sn["end_date"],[sn["start_date"],sn["end_date"]])
    votable.to_xml("tmp/new_votable.xml")


    
    
params = {'claimedtype' : 'Ia', 'format' : 'csv' }
Catalog_domain ='https://api.sne.space/catalog'
query_str  = urlencode(params)
url = '{}?{}&first'.format(Catalog_domain,query_str)
sn_data = pd.read_csv(url)


print("set constraints on the supernova catalog table")
z_min, z_max = 0.2, 0.3
sn_data = sn_data[(sn_data['redshift']>z_min) & (sn_data['redshift']<z_max)] 
sn_data = sn_data[sn_data['event'].str.startswith('SNLS')]
sn_data = sn_data[sn_data['discoverdate']< sn_data['maxdate']]


sn_data["ra"] =SkyCoord(sn_data["ra"], sn_data["dec"],unit=(u.hourangle,u.deg)).ra.degree
sn_data["dec"] = SkyCoord(sn_data["ra"], sn_data["dec"],unit=(u.hourangle,u.deg)).dec.degree

days_before = 20.0
days_after = 100.0
sn_data["maxdate"] = sn_data["maxdate"].str.replace('/','-')

print("Calculating coords and start end date")
for index, row in sn_data.iterrows():
    if(str(row["maxdate"])!='nan'):
        processed_maxdate = Time(row["maxdate"], format='isot')
        sn_data.at[index, 'start_date'] =  (processed_maxdate - TimeDelta(days_before,format='jd')).mjd  
        sn_data.at[index, 'end_date'] = (processed_maxdate + TimeDelta(days_after, format='jd')).mjd
    else: ##set place holder 
        sn_data.at[index, 'start_date'] =  0.0  
        sn_data.at[index, 'end_date'] = 0.1       
        

sn_data.dropna(subset=['maxdate']) 
        
field_dictionary = {
    "event":["char", "*"],
    "alias":["char", "*"],
    "catalog":["char", "*"],
    "claimedtype":["char", "*"],
    "dec":"double",
    "maxdate" :["char","*"], 
    "name" :["char","*"],
    "ra" :"double" ,
    "start_date":"double",
    "end_date": "double",
    "start_end_date":["double", "2", "interval"]   
}

### create a votable file
create_a_new_votable(field_dictionary, sn_data)
###

#52865.0,52985.0
#334.0276124999999, -17.704655555555554
anonSubject = net.Subject()
client = CadcTapClient(anonSubject, resource_id ='ivo://cadc.nrc.ca/tap')
query ="""SELECT * FROM caom2.Plane AS Plane
    JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID join tap_upload.tmptable as Tmp on
    (INTERSECTS(Tmp.start_end_date, Plane.time_bounds_samples) = 1
    AND CONTAINS( POINT('ICRS',Tmp.ra, Tmp.dec ), Plane.position_bounds ) = 1)
    WHERE (LOWER(Plane.energy_bandpassName) LIKE 'r%'
    AND name ='SNLS-03D4cj'
    AND collection = 'CFHT'
    AND calibrationLevel >= 2
    AND (Plane.quality_flag IS NULL OR Plane.quality_flag != 'junk'))
    ORDER BY time_bounds_lower ASC"""

def_table = os.path.join('tmp','new_votable.xml')
print("fetching data from the database")
client.query(query, output_file= "tmp/out_file.csv", response_format= 'csv', tmptable='tmptable:' + def_table, timeout = 30)

#client.query(query, response_format= 'csv', tmptable='tmptable:' + def_table)
    
   
app = gj.jglue()
out_data = app.load_data('tmp/out_file.csv')

scatter_viewer = app.scatter2d(x = 'time_bounds_lower',y = 'time_exposure', data = out_data)
histogram_viewer = app.histogram1d(x = 'time_exposure', data = out_data)

set constraints on the supernova catalog table
Calculating coords and start end date
Creating the votable file for the tmp table.
fetching data from the database
